In [11]:
!pip install -qqq torch pandas numpy pytorch_lightning sentence-transformers

In [16]:
# Standard libraries
import os
import csv

# Standard data processing libraries
import pandas as pd
import numpy as np

# ML libraries
import torch
from torch.utils.data import Dataset
from pytorch_lightning import Trainer

# Project code
from lib.dataset import SBERTDataset
from lib.data_module import SBERTDataModule
from lib.model import SBERT_iSTS_Model


In [18]:
PROJECT_DIR = "./" # @param {type: "string"}
DATASET = "headlines" # @param {type: "string"}
DATA_DIR = os.path.join(PROJECT_DIR, "data/sem_eval_2016/", DATASET)

TRAIN_BATCH_SIZE = 16 # @param {type: "slider", min:1, max:128}
BATCH_SIZE = 16 # @param {type: "slider", min:1, max:128}
NUM_WORKERS = 2 # @param {type: "slider", min:1, max:16}

EPOCHS = 10 # @param {type: "slider", min:1, max:128}
ACCELERATOR = "auto" # @param ["auto", "gpu", "tpu", "cpu"]

In [14]:
model = SBERT_iSTS_Model()

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 839kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 239kB/s]
Downloading: 100%|██████████| 10.6k/10.6k [00:00<00:00, 6.36MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 75.7kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 290kB/s] 
Downloading: 100%|██████████| 438M/438M [00:19<00:00, 22.9MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 45.9kB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 297kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 704kB/s]  
Downloading: 100%|██████████| 363/363 [00:00<00:00, 291kB/s]
Downloading: 100%|██████████| 13.1k/13.1k [00:00<00:00, 7.16MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 447kB/s]  
Downloading: 100%|██████████| 349/349 [00:00<00:00, 252kB/s]


In [19]:
data = SBERTDataModule(
    f'{DATA_DIR}/train.tsv', 
    f'{DATA_DIR}/test.tsv', 
    batch_size=BATCH_SIZE, 
    train_batch_size=TRAIN_BATCH_SIZE,
    num_workers=NUM_WORKERS
)

In [ ]:
trainer = Trainer(a)